In [ ]:
import pandas as pd 

In [1]:
import pandas as pd

In [9]:
df = pd.read_csv(r'C:\\Users\\Ndiaye\\Documents\\2_MASTER_2\\1e_SEMESTRE_M2\\Streamlit\\OpenData\\DATA\\commune_500.csv', encoding='latin1')
df.head()

,objectid,id,nom,nom_m,insee_com,statut,population,insee_can,insee_arr,insee_dep,insee_reg,siren_epci,gdb_geomattr_data,shape
0,1,COMMUNE_0000000009760754,Lourties-Monbrun,LOURTIES-MONBRUN,32216,Commune simple,162.0,03,3.0,32,76.0,200072320,NaN,01030000206A080000010000002900000080999999E9A6...
1,2,COMMUNE_0000000009756744,Boudy-de-Beauregard,BOUDY-DE-BEAUREGARD,47033,Commune simple,402.0,10,3.0,47,75.0,200036523,NaN,01030000206A080000010000002C000000809999999B73...
2,3,COMMUNE_0000000009754403,Autrans-Méaudre en Vercors,AUTRANS-MEAUDRE EN VERCORS,38225,Commune simple,3075.0,07,1.0,38,84.0,243801024,NaN,01030000206A0800000100000067000000403333330C60...
3,4,COMMUNE_0000000009760252,Armous-et-Cau,ARMOUS-ET-CAU,32009,Commune simple,93.0,16,3.0,32,76.0,243200425,NaN,01030000206A080000010000002E000000000000009EFB...
4,5,COMMUNE_0000000009727903,Willeman,WILLEMAN,62890,Commune simple,182.0,06,4.0,62,32.0,200044030,NaN,01030000206A0800000100000031000000000000008096...


# Étape 1 : Filtrage des communes du Gard et de l’Hérault

In [13]:
import geopandas as gpd
# Filtrer pour le Gard (30) et l’Hérault (34)
df_filtrer = df[df["insee_dep"].isin(["30", "34"])]


In [14]:
df_filtrer.head()

,objectid,id,nom,nom_m,insee_com,statut,population,insee_can,insee_arr,insee_dep,insee_reg,siren_epci,gdb_geomattr_data,shape
58,58,COMMUNE_0000000009757626,Le Martinet,LE MARTINET,30159,Commune simple,759.0,18,1.0,30,76.0,200066918,NaN,01030000206A080000010000003400000080999999ECF0...
195,188,COMMUNE_0000000009760641,Margon,MARGON,34149,Commune simple,736.0,05,1.0,34,76.0,200071058,NaN,01030000206A0800000100000015000000C0CCCCCC8D16...
205,199,COMMUNE_0000000009760913,Cassagnoles,CASSAGNOLES,34054,Commune simple,110.0,24,1.0,34,76.0,200066348,NaN,01030000206A080000010000003A00000080999999F049...
230,223,COMMUNE_0000000009760389,Riols,RIOLS,34229,Commune simple,721.0,24,1.0,34,76.0,200066348,NaN,01030000206A0800000100000052000000000000007EDA...
238,233,COMMUNE_0000000009758785,Campestre-et-Luc,CAMPESTRE-ET-LUC,30064,Commune simple,145.0,22,3.0,30,76.0,243000270,NaN,01030000206A080000010000004400000000000000567E...


# Etape2 :  Nettoyage des colonnes inutiles et correction des formats de colonnes(changement des type)

In [16]:
# seulement les colonnes utiles
colonnes_utilies = ['insee_com','nom', 'insee_dep', 'insee_reg', 'siren_epci', 'population', 'shape']
df_clean = df_filtrer[colonnes_utilies].copy()

# Conversion  des colonnes en entiers(int)
df_clean['population'] = df_clean['population'].astype(int)
df_clean['insee_dep'] = df_clean['insee_dep'].astype(int)
df_clean['insee_reg'] = df_clean['insee_reg'].astype(int)


# Etape 3 :  Conversion de la colonne shape en géométries

In [ ]:
from shapely.wkb import loads
import geopandas as gpd

# Conversion de la colonne 'shape' en géométries
df_clean['geometry'] = df_clean['shape'].apply(lambda x: loads(x, hex=True))

# GeoDataFrame
gdf_clean = gpd.GeoDataFrame(df_clean, geometry='geometry', crs="EPSG:4326")

# Suppression de la colonne 'shape' (plus necessaire)
# gdf_clean = gdf_clean.drop(columns=['shape'])
gdf_clean.head()

,insee_com,nom,insee_dep,insee_reg,siren_epci,population,shape,geometry
58,30159,Le Martinet,30,76,200066918,759,01030000206A080000010000003400000080999999ECF0...,"POLYGON ((784502.3 6350275.5, 784512 6350284.5..."
195,34149,Margon,34,76,200071058,736,01030000206A0800000100000015000000C0CCCCCC8D16...,"POLYGON ((723782.9 6264082.3, 723855.2 6264189..."
205,34054,Cassagnoles,34,76,200066348,110,01030000206A080000010000003A00000080999999F049...,"POLYGON ((664824.3 6258276.2, 664947.3 6258359..."
230,34229,Riols,34,76,200066348,721,01030000206A0800000100000052000000000000007EDA...,"POLYGON ((683327 6272556, 683349.3 6272512.4, ..."
238,30064,Campestre-et-Luc,30,76,243000270,145,01030000206A080000010000004400000000000000567E...,"POLYGON ((737067 6314108.9, 737085.7 6314091.2..."


In [21]:
# pip install openpyxl

In [22]:
# gdf_clean.to_file("../DATA/communes_30_34_clean.geojson", driver="GeoJSON")
# gdf_clean.drop(columns=['geometry']).to_excel("../DATA/communes_30_34_clean.xlsx", index=False) #pour export en xlsx 


# Etape 4 : Chargement et filtre les données CC Logement

In [ ]:
df_cc_logement= pd.read_csv("C:\\Users\\Ndiaye\\Documents\\2_MASTER_2\\1e_SEMESTRE_M2\\Streamlit\\OpenData\\SORTIE\\Compil_clean.csv")
df_cc_logement.head()


,CODGEO,LIBGEO,DEP,LOG,RP,RSECOCC,LOGVAC,MAISON,APPART,RP_1P,...,RP_ACH05,RP_ACH18,RP_ACH17,RP_ACH15,RP_ACH14,RP_ACH13,RP_ACH12,RP_ACH11,RP_ACH10,RP_LOCPRIV
0,30001,Aigaliers,30,350.745416,229.895704,89.912186,30.937526,329.380299,19.413995,4.875347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.213457
1,30002,Aigremont,30,414.761075,327.900205,77.663837,9.197033,390.869741,20.817543,4.165367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.421945
2,30003,Aigues-Mortes,30,5376.776608,3983.513458,1084.436653,308.826498,3810.180887,1527.882255,73.796932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1032.117277
3,30004,Aigues-Vives,30,1628.669871,1417.886360,90.191022,120.592489,1424.993904,197.521667,24.947336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.631063
4,30005,Aiguèze,30,250.965647,110.032157,113.718747,27.214743,188.938835,54.307282,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.116791


In [26]:
df_cc_logement

,CODGEO,LIBGEO,DEP,LOG,RP,RSECOCC,LOGVAC,MAISON,APPART,RP_1P,...,RP_ACH05,RP_ACH18,RP_ACH17,RP_ACH15,RP_ACH14,RP_ACH13,RP_ACH12,RP_ACH11,RP_ACH10,RP_LOCPRIV
0,30001,Aigaliers,30,350.745416,229.895704,89.912186,30.937526,329.380299,19.413995,4.875347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.213457
1,30002,Aigremont,30,414.761075,327.900205,77.663837,9.197033,390.869741,20.817543,4.165367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.421945
2,30003,Aigues-Mortes,30,5376.776608,3983.513458,1084.436653,308.826498,3810.180887,1527.882255,73.796932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1032.117277
3,30004,Aigues-Vives,30,1628.669871,1417.886360,90.191022,120.592489,1424.993904,197.521667,24.947336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.631063
4,30005,Aiguèze,30,250.965647,110.032157,113.718747,27.214743,188.938835,54.307282,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.116791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6926,34340,Villetelle,34,556.128635,526.812845,19.206897,10.108893,522.011231,32.110497,2.006906,...,264.911602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.193370,99.341851
6927,34341,Villeveyrac,34,1615.474093,1413.000000,66.113990,136.360104,1489.783344,118.507599,6.146641,...,308.701650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208.416864,325.269999
6928,34342,Viols-en-Laval,34,101.190476,87.857143,7.777778,5.555556,77.862616,23.327860,0.000000,...,34.334975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.078818,29.285714
6929,34343,Viols-le-Fort,34,539.000000,488.000000,16.000000,35.000000,442.000000,72.000000,10.000000,...,126.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.000000,138.000000


In [25]:
# Filtre pour l'année 2022
df_cc_2022 = df_cc_logement_22[df_cc_logement_22['AN'] == 2022]
df_cc_2022.head()

,CODGEO,LIBGEO,DEP,LOG,RP,RSECOCC,LOGVAC,MAISON,APPART,RP_1P,...,RP_ACH05,RP_ACH18,RP_ACH17,RP_ACH15,RP_ACH14,RP_ACH13,RP_ACH12,RP_ACH11,RP_ACH10,RP_LOCPRIV
0,30001,Aigaliers,30,350.745416,229.895704,89.912186,30.937526,329.380299,19.413995,4.875347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.213457
1,30002,Aigremont,30,414.761075,327.900205,77.663837,9.197033,390.869741,20.817543,4.165367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.421945
2,30003,Aigues-Mortes,30,5376.776608,3983.513458,1084.436653,308.826498,3810.180887,1527.882255,73.796932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1032.117277
3,30004,Aigues-Vives,30,1628.669871,1417.886360,90.191022,120.592489,1424.993904,197.521667,24.947336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.631063
4,30005,Aiguèze,30,250.965647,110.032157,113.718747,27.214743,188.938835,54.307282,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.116791


In [28]:
# Renommer CODGEO en insee_com pour la jointure
df_cc_2022 = df_cc_2022.rename(columns={'CODGEO': 'insee_com'})
df_cc_2022.head()

,insee_com,LIBGEO,DEP,LOG,RP,RSECOCC,LOGVAC,MAISON,APPART,RP_1P,...,RP_ACH05,RP_ACH18,RP_ACH17,RP_ACH15,RP_ACH14,RP_ACH13,RP_ACH12,RP_ACH11,RP_ACH10,RP_LOCPRIV
0,30001,Aigaliers,30,350.745416,229.895704,89.912186,30.937526,329.380299,19.413995,4.875347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.213457
1,30002,Aigremont,30,414.761075,327.900205,77.663837,9.197033,390.869741,20.817543,4.165367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.421945
2,30003,Aigues-Mortes,30,5376.776608,3983.513458,1084.436653,308.826498,3810.180887,1527.882255,73.796932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1032.117277
3,30004,Aigues-Vives,30,1628.669871,1417.886360,90.191022,120.592489,1424.993904,197.521667,24.947336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.631063
4,30005,Aiguèze,30,250.965647,110.032157,113.718747,27.214743,188.938835,54.307282,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.116791


# Etape 5 : Jointure avec les données géographiques

In [32]:
# Convertir 'insee_com' en str dans les deux DataFrames
gdf_clean['insee_com'] = gdf_clean['insee_com'].astype(str)
df_cc_2022['insee_com'] = df_cc_2022['insee_com'].astype(str)


In [34]:
gdf_final = gdf_clean.merge(df_cc_2022, on='insee_com', how='left')



In [35]:
gdf_final.head()

,insee_com,nom,insee_dep,insee_reg,siren_epci,population,shape,geometry,LIBGEO,DEP,...,RP_ACH05,RP_ACH18,RP_ACH17,RP_ACH15,RP_ACH14,RP_ACH13,RP_ACH12,RP_ACH11,RP_ACH10,RP_LOCPRIV
0,30159,Le Martinet,30,76,200066918,759,01030000206A080000010000003400000080999999ECF0...,"POLYGON ((784502.3 6350275.5, 784512 6350284.5...",Le Martinet,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131.072019
1,34149,Margon,34,76,200071058,736,01030000206A0800000100000015000000C0CCCCCC8D16...,"POLYGON ((723782.9 6264082.3, 723855.2 6264189...",Margon,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.907514
2,34054,Cassagnoles,34,76,200066348,110,01030000206A080000010000003A00000080999999F049...,"POLYGON ((664824.3 6258276.2, 664947.3 6258359...",Cassagnoles,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.881818
3,34229,Riols,34,76,200066348,721,01030000206A0800000100000052000000000000007EDA...,"POLYGON ((683327 6272556, 683349.3 6272512.4, ...",Riols,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.027405
4,30064,Campestre-et-Luc,30,76,243000270,145,01030000206A080000010000004400000000000000567E...,"POLYGON ((737067 6314108.9, 737085.7 6314091.2...",Campestre-et-Luc,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.552239


In [37]:
gdf_final[['nom', 'insee_com', 'RP', 'LOG']].head()


,nom,insee_com,RP,LOG
0,Le Martinet,30159,385.713473,594.007112
1,Margon,34149,312.411621,404.752870
2,Cassagnoles,34054,61.436364,107.436364
3,Riols,34229,363.519822,666.513124
4,Campestre-et-Luc,30064,78.783582,144.990479


In [39]:
gdf_final.to_file("../DATA/communes_30_34_with_cc_2022.geojson", driver="GeoJSON")


In [42]:
# pip install folium

In [46]:
import folium
from folium.plugins import MarkerCluster

# Créer une carte centrée sur la région
m = folium.Map(location=[43.8, 4.0], zoom_start=8)

# Ajouter les communes avec une couleur basée sur le taux de logements (LOG)
folium.GeoJson(
    gdf_final,
    tooltip=folium.GeoJsonTooltip(fields=['nom', 'LOG', 'RP']),
    style_function=lambda feature: {
        'fillColor': 'blue' if feature['properties']['LOG'] > 100 else 'green',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6
    }
).add_to(m)




In [48]:
# Sauvegarder la carte
# m.save("carte_communes_2022.html")